<a href="https://colab.research.google.com/github/Nareshedagotti/RAG/blob/main/Standard_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
pip install langchain groq faiss-cpu sentence-transformers PyPDF2 python-docx langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 950.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
#text extraction
from PyPDF2 import PdfReader
from docx import Document

def extract_text(file_paths):
    documents = []
    for file_path in file_paths:
        if file_path.endswith('.pdf'):
            with open(file_path, 'rb') as file:
                pdf_reader = PdfReader(file)
                text = "".join(page.extract_text() or "" for page in pdf_reader.pages)
                documents.append({"content": text, "source": file_path})
        elif file_path.endswith('.docx'):
            doc = Document(file_path)
            text = "\n".join(para.text for para in doc.paragraphs)
            documents.append({"content": text, "source": file_path})
    return documents

file_path = ["/content/How Vectors - 1_merged.pdf" ]
text = extract_text(file_path)
print(text)

[{'content': 'HOW \nVECTORS ARE STORED,\n INDEXED, AND RETRIEVED\nINTRODUCTION\nHow text is converted into vectors, stored in a database, \nindexed efficiently, and used to retrieve similar \ninformation — with code & real-world examples.\nSTATFUSIONAI"Let\'s go deep behind the scenes."\nTEXT TO VECTOR – EMBEDDING\nCode:\nOutput Example:\nfrom sentence_transformers import SentenceTransformer\nmodel = SentenceTransformer(\'all-MiniLM-L6-v2\')\ntexts = [\n    "Apples are sweet and red.",\n    "Bananas are yellow and soft.",\n    "Kids love mangoes during summer."\n]\nembeddings = model.encode(texts)\nBehind the Scenes: \n•When model.encode() runs, it:\n1.Tokenizes text → converts to word pieces (subwords like \n"embed" + "##ding")\n2.Feeds tokens through transformer layers with attention \nmechanisms\n3.Pools token representations to create a single vector\n4.Outputs a vector (384 dimensions) representing semantic \nmeaning\nprint(embeddings[0][:5])  # First 5 values of first embedding\n

In [7]:
#Splitting text into chunkings
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunk_text(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    chunks = []
    for doc in documents:
        split_docs = text_splitter.create_documents(
            [doc["content"]],
            metadatas=[{"source": doc["source"]}]
        )
        chunks.extend(split_docs)
    return chunks

chunks = chunk_text(text)
for i, chunk in enumerate(chunks, 1):
    print(f"\n🔹 **Chunk {i}:**\n{chunk.page_content}\n{'-'*50}")


🔹 **Chunk 1:**
HOW 
VECTORS ARE STORED,
 INDEXED, AND RETRIEVED
INTRODUCTION
How text is converted into vectors, stored in a database, 
indexed efficiently, and used to retrieve similar 
information — with code & real-world examples.
STATFUSIONAI"Let's go deep behind the scenes."
TEXT TO VECTOR – EMBEDDING
Code:
Output Example:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
texts = [
    "Apples are sweet and red.",
    "Bananas are yellow and soft.",
    "Kids love mangoes during summer."
]
embeddings = model.encode(texts)
Behind the Scenes: 
•When model.encode() runs, it:
1.Tokenizes text → converts to word pieces (subwords like 
"embed" + "##ding")
2.Feeds tokens through transformer layers with attention 
mechanisms
3.Pools token representations to create a single vector
4.Outputs a vector (384 dimensions) representing semantic 
meaning
print(embeddings[0][:5])  # First 5 values of first embedding
# [0.1213, 0.3421, -0.5632, 0.

In [ ]:
#Converting Chunks into embeddings
from langchain.embeddings import HuggingFaceEmbeddings

def generate_embeddings(chunks):
    embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
    return embedding_model, chunks

embedding_model, chunks = generate_embeddings(chunks)

In [ ]:
#storing embeddings in Faiss vectordb
from langchain.vectorstores import FAISS

def store_in_db(chunks, embedding_model):
    vector_db = FAISS.from_documents(chunks, embedding_model)
    return vector_db

vector_db = store_in_db(chunks, embedding_model)

In [ ]:
#converting query into embeddings
def query_to_embeddings(query, embedding_model):
    query_embedding = embedding_model.embed_query(query)
    return query_embedding

query = "give me different types of vector indexing and uses?"
query_embedding = query_to_embeddings(query, embedding_model)

In [ ]:
#retrieving top matched context from vectordb
def retrieve_top_matches(vector_db, query_embedding):
    top_matches = vector_db.similarity_search_by_vector(query_embedding, k=3)
    return top_matches

top_matches = retrieve_top_matches(vector_db, query_embedding)
print(top_matches)

In [ ]:
#sending retrieved context to llm
from groq import Groq

def pass_to_llm(query, top_matches, api_key):
    context = "\n\n".join(match.page_content for match in top_matches)
    prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    client = Groq(api_key=api_key)
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model="gemma2-9b-it"
    )
    return chat_completion.choices[0].message.content, top_matches


api_key="your_api_key"
llm_response, top_matches = pass_to_llm(query, top_matches, api_key)
print(llm_response)